## Setup PyDrake using underactuated in Google Colab

In [1]:
import sys
import importlib
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

## Import Libraries

In [2]:
# Imports
import os
import numpy as np

from pydrake.all import MultibodyPlant, Parser

from pydrake.all import DirectTranscription, DirectCollocation, Solve

from pydrake.autodiffutils import AutoDiffXd

## Setup URDF Model Path

In [3]:
if 'google.colab' in sys.modules:
  # Check if this cell has already been run:
  if not os.path.isdir('DrakeTests/boxTrajOpt/'):
    !git clone https://github.com/vyas-shubham/DrakeTests.git
  urdfFolderPath = 'DrakeTests/boxTrajOpt/'
else:
  urdfFolderPath = ''

# Comment/Uncomment the lines below to select a URDF
urdfpath = urdfFolderPath+'box.urdf'
assert os.path.isfile(urdfpath)

## Optimization Problem

The current problem is taken from **Betts, J. T. (2010). Practical Methods for Optimal Control and Estimation Using Nonlinear Programming.** 

**Example 6.12: Reorientation of a Rigid Body.**

The urdf contains the inertias as given in the book: Ixx = 5621, Iyy = 4547, Izz = 2364

Torque Limits = abs(u) <= 50Nm

$\phi = 150^o$

Initial State: Rotation Quaternion q(0) = [1,0,0,0]. Position Vector = [0,0,0]

Final State: Rotation Quaternion q(N) = $[cos(\phi/2),sin(\phi/2),0,0]$. Position Vector = [0,0,0]

The trajectory optimisation is setup using generalized forces as the inputs. No actuators are used.

Since this is similar to a double integrator in rotation form, the optimal solution is a bang-bang control with max/min torques being applied to each of the rotation axes. The time taken (for one of the solutions) is 28.63s. 

### Initial and Final State Arrays

In [4]:
x0_base_position = [1, 0, 0, 0, 0, 0, 0]
x0_base_vel = [0, 0, 0, 0, 0, 0]
x0 = np.array(x0_base_position + x0_base_vel)

phi = np.deg2rad(150)
xf_base_position = [np.cos(phi/2), np.sin(phi/2), 0, 0, 0, 0, 0]
xf_base_vel = [0, 0, 0, 0, 0, 0]
xf = np.array(xf_base_position + xf_base_vel)

### Using Direct Transciption

The solver is not able to find a solution to due infeasible constraints.

In [16]:
# # Direct Transcription Method

dt = 0.1

plant_d = MultibodyPlant(time_step=dt)
box_instance_d = Parser(plant_d).AddModelFromFile(urdfpath)
# Remove gravity
plantGravityField = plant_d.gravity_field()
plantGravityField.set_gravity_vector([0,0,0])
plant_d.Finalize()
# plant_autodiff = plant_d.ToAutoDiffXd()

trajOptContext = plant_d.CreateDefaultContext()

NDirTrans = 300


prog = DirectTranscription(plant_d, trajOptContext, NDirTrans,
                           input_port_index=plant_d.get_applied_generalized_force_input_port().get_index())

# for x_index in range(x0.size):
#     prog.AddConstraint(prog.initial_state()[x_index] == x0[x_index])
#     prog.AddConstraint(prog.final_state()[x_index] == xf[x_index])

prog.AddBoundingBoxConstraint(x0, x0, prog.initial_state())
prog.AddBoundingBoxConstraint(xf, xf, prog.final_state())

for u_index in range(prog.input().size):
    prog.AddConstraintToAllKnotPoints(prog.input()[u_index] <= 50)
    prog.AddConstraintToAllKnotPoints(prog.input()[u_index] >= 50)


    
result = Solve(prog)

print("Optimisation Solution Found?: {}".format(result.is_success()))

if not result.is_success():
    infeasible = result.GetInfeasibleConstraints(prog)
    print("Infeasible constraints:")
    for i in range(len(infeasible)):
        print(infeasible[i])

Optimisation Solution Found?: False
Infeasible constraints:
DirectTranscriptionConstraint with 32 decision variables u(0) u(1) u(2) u(3) u(4) u(5) x(0) x(1) x(2) x(3) x(4) x(5) x(6) x(7) x(8) x(9) x(10) x(11) x(12) x(13) x(14) x(15) x(16) x(17) x(18) x(19) x(20) x(21) x(22) x(23) x(24) x(25)
DirectTranscriptionConstraint with 32 decision variables u(0) u(1) u(2) u(3) u(4) u(5) x(0) x(1) x(2) x(3) x(4) x(5) x(6) x(7) x(8) x(9) x(10) x(11) x(12) x(13) x(14) x(15) x(16) x(17) x(18) x(19) x(20) x(21) x(22) x(23) x(24) x(25)
DirectTranscriptionConstraint with 32 decision variables u(0) u(1) u(2) u(3) u(4) u(5) x(0) x(1) x(2) x(3) x(4) x(5) x(6) x(7) x(8) x(9) x(10) x(11) x(12) x(13) x(14) x(15) x(16) x(17) x(18) x(19) x(20) x(21) x(22) x(23) x(24) x(25)
DirectTranscriptionConstraint with 32 decision variables u(0) u(1) u(2) u(3) u(4) u(5) x(0) x(1) x(2) x(3) x(4) x(5) x(6) x(7) x(8) x(9) x(10) x(11) x(12) x(13) x(14) x(15) x(16) x(17) x(18) x(19) x(20) x(21) x(22) x(23) x(24) x(25)
DirectTr

DirectTranscriptionConstraint with 32 decision variables u(876) u(877) u(878) u(879) u(880) u(881) x(1898) x(1899) x(1900) x(1901) x(1902) x(1903) x(1904) x(1905) x(1906) x(1907) x(1908) x(1909) x(1910) x(1911) x(1912) x(1913) x(1914) x(1915) x(1916) x(1917) x(1918) x(1919) x(1920) x(1921) x(1922) x(1923)
DirectTranscriptionConstraint with 32 decision variables u(876) u(877) u(878) u(879) u(880) u(881) x(1898) x(1899) x(1900) x(1901) x(1902) x(1903) x(1904) x(1905) x(1906) x(1907) x(1908) x(1909) x(1910) x(1911) x(1912) x(1913) x(1914) x(1915) x(1916) x(1917) x(1918) x(1919) x(1920) x(1921) x(1922) x(1923)
DirectTranscriptionConstraint with 32 decision variables u(876) u(877) u(878) u(879) u(880) u(881) x(1898) x(1899) x(1900) x(1901) x(1902) x(1903) x(1904) x(1905) x(1906) x(1907) x(1908) x(1909) x(1910) x(1911) x(1912) x(1913) x(1914) x(1915) x(1916) x(1917) x(1918) x(1919) x(1920) x(1921) x(1922) x(1923)
DirectTranscriptionConstraint with 32 decision variables u(876) u(877) u(878) u

DirectTranscriptionConstraint with 32 decision variables u(1716) u(1717) u(1718) u(1719) u(1720) u(1721) x(3718) x(3719) x(3720) x(3721) x(3722) x(3723) x(3724) x(3725) x(3726) x(3727) x(3728) x(3729) x(3730) x(3731) x(3732) x(3733) x(3734) x(3735) x(3736) x(3737) x(3738) x(3739) x(3740) x(3741) x(3742) x(3743)
DirectTranscriptionConstraint with 32 decision variables u(1716) u(1717) u(1718) u(1719) u(1720) u(1721) x(3718) x(3719) x(3720) x(3721) x(3722) x(3723) x(3724) x(3725) x(3726) x(3727) x(3728) x(3729) x(3730) x(3731) x(3732) x(3733) x(3734) x(3735) x(3736) x(3737) x(3738) x(3739) x(3740) x(3741) x(3742) x(3743)
DirectTranscriptionConstraint with 32 decision variables u(1716) u(1717) u(1718) u(1719) u(1720) u(1721) x(3718) x(3719) x(3720) x(3721) x(3722) x(3723) x(3724) x(3725) x(3726) x(3727) x(3728) x(3729) x(3730) x(3731) x(3732) x(3733) x(3734) x(3735) x(3736) x(3737) x(3738) x(3739) x(3740) x(3741) x(3742) x(3743)
DirectTranscriptionConstraint with 32 decision variables u(17

### Using Direct Collocation

The solver is not able to find a solution to due infeasible constraints.

In [5]:
# Direct Collocation Method

plant = MultibodyPlant(time_step=0.0)
box_instance = Parser(plant).AddModelFromFile(urdfpath)
# Remove gravity
plantGravityField = plant.gravity_field()
plantGravityField.set_gravity_vector([0,0,0])
plant.Finalize()
# plant = plant.ToAutoDiffXd()
trajOptContext = plant.CreateDefaultContext()

NDirCol = 60
h_min = 0.01
h_max = 0.5

dircol = DirectCollocation(plant,
                           trajOptContext,
                           num_time_samples=NDirCol,
                           minimum_timestep=h_min,
                           maximum_timestep=h_max,
                           input_port_index=plant.get_applied_generalized_force_input_port().get_index())

dircol.AddEqualTimeIntervalsConstraints()

# for x_index in range(x0.size):
#     dircol.AddConstraint(dircol.initial_state()[x_index] == x0[x_index])
#     dircol.AddConstraint(dircol.final_state()[x_index] == xf[x_index])

dircol.AddBoundingBoxConstraint(x0, x0, dircol.initial_state())
dircol.AddBoundingBoxConstraint(xf, xf, dircol.final_state())
    
for u_index in range(dircol.input().size):
    dircol.AddConstraintToAllKnotPoints(dircol.input()[u_index] <= 50)
    dircol.AddConstraintToAllKnotPoints(dircol.input()[u_index] >= -50)

# Add a final cost equal to the total duration.
dircol.AddFinalCost(dircol.time())

result = Solve(dircol)

print("Optimisation Solution Found?: {}".format(result.is_success()))

if not result.is_success():
    infeasible = result.GetInfeasibleConstraints(prog)
    print("Infeasible constraints:")
    for i in range(len(infeasible)):
        print(infeasible[i])

RuntimeError: Encountered singular articulated body hinge inertia for body node index 1. Please ensure that this body has non-zero inertia along all axes of motion.

Verification that the Body at Body Node Index 1 has non-zero inertia along all axes of motion.

In [9]:
body1_index = plant.GetBodyIndices(box_instance)[0]
body1 = plant.get_body(body1_index)
print(body1.index())
print(body1.default_spatial_inertia().CopyToFullMatrix6())

BodyIndex(1)
[[5621.    0.    0.    0.   -0.    0.]
 [   0. 4547.    0.    0.    0.   -0.]
 [   0.    0. 2364.   -0.    0.    0.]
 [  -0.    0.   -0.  100.    0.    0.]
 [  -0.   -0.    0.    0.  100.    0.]
 [   0.   -0.   -0.    0.    0.  100.]]
